<b>Setup</b> 

In [ ]:
# activate the local environment
import Pkg
Pkg.activate(".")
# Instantiate only necessary when running for the first time (will download and install required packages)
 #Pkg.instantiate() 

In [ ]:
#Load necessary packages
using CSV
using DataFrames
using JuMP
using LinearAlgebra
using Random
using Plots
using SparseArrays
using XLSX
using SCS
using Distributions
using HTTP
using Dates
using Printf

<b>Setup system data</b> 

In [ ]:
#Setup system data
#ieee_33buses_dataset
function read_csv_from_github(url::String)
    response = HTTP.get(url)
    return CSV.read(IOBuffer(response.body), DataFrame)
end

file_loc_gen = "Data/33_bus_dataset/Generator.csv"
file_loc_line = "Data/33_bus_dataset/Line.csv"
file_loc_node = "Data/33_bus_dataset/Node_full.csv"
file_loc_pv = "Data/33_bus_dataset/pvveryhigh.csv"
file_loc_bat = "Data/33_bus_dataset/battery.csv"

dfg = CSV.read(file_loc_gen, DataFrame)
dfl = CSV.read(file_loc_line, DataFrame)
dfb = CSV.read(file_loc_node, DataFrame)
dfpv = CSV.read(file_loc_pv, DataFrame)
dfbat = CSV.read(file_loc_bat, DataFrame)

<b>Load dataset</b> 

In [ ]:
#Load dataset
file_loc_load = "Data/Load_data/time_series_60min_singleindex.csv"
df = CSV.read(file_loc_load, DataFrame)

<b>Define the normalization function</b> 

In [ ]:
function normalize_column!(col::AbstractVector{T}) where T<:Real
    min_val = minimum(col)
    max_val = maximum(col)
    return (col) / (max_val)
end
#Apply the normalization to each numeric column
for col_name in names(df)
    if eltype(df[!, col_name]) <: Real
        df[!, col_name] = normalize_column!(df[!, col_name])
    end
end
#Define the format of your timestamps
dateformat = "yyyy-mm-dd\\THH:MM:SSZ"
#Filter the DataFrame to include only the rows for the 24 hours
start_time = DateTime("2015-01-02T00:00:00Z", dateformat)
end_time = DateTime("2015-01-03T00:00:00Z", dateformat)
df_24h = filter(row -> start_time <= DateTime(row[:utc_timestamp], dateformat) < end_time, df)
#Define voltage base and Sbase 
Vbase = 12.66 * 1e3;  # in Volts#basekV=12.66
Sbase = 1. * 1e6;  #in VA #baseMVA=1
num_buses = 33


cum24hourseachbus = []
for i in 1:num_buses
    eachbus = df_24h.AT_load_actual_entsoe_transparency.* dfb[!,"Pd"][i].* 1e3.*0.5 ./ Sbase
    push!(cum24hourseachbus,eachbus)
end

cumbuseachhour = []
for i in 1:24
    eachhour = df_24h.AT_load_actual_entsoe_transparency[i].* dfb[!,"Pd"].* 1e3.*0.5 ./ Sbase
    push!(cumbuseachhour,eachhour)
end    

cumbuseachhourreactiveload = []
for i in 1:24
    eachhourreactiveload = df_24h.AT_load_actual_entsoe_transparency[i].* dfb[!,"Qd"].* 1e3.*0.5 ./ Sbase
    push!(cumbuseachhourreactiveload,eachhourreactiveload )
end  

<b>PV dataset</b> 

In [ ]:
data_pv = "Data/PVdataset/pvdataset.csv"
smart_home_data = CSV.read(data_pv,DataFrame)
smart_home_data[!, :local_15min] = DateTime.(smart_home_data[!, :local_15min], "m/d/y H:M")
#Define the format of the timestamps
dateformat = "m/d/y H:M"
#Filter the DataFrame to include only the rows for the 24-hour period
start_time = DateTime("3/2/15 0:0", dateformat)
end_time = DateTime("3/3/15 0:0", dateformat)
df_24h = filter(row -> start_time <= row[:local_15min] < end_time, smart_home_data)
#Filter to get only the start of each hour
df_hourly = df_24h[minute.(df_24h[!, :local_15min]) .== 0, :]


In [ ]:
function normalize_column!(col::AbstractVector{T}) where T<:Real
    min_val = minimum(col)
    max_val = maximum(col)
    return (col) / (max_val)
end
#Apply the normalization to each numeric column
for col_name in names( df_hourly)
    if eltype( df_hourly[!, col_name]) <: Real
         df_hourly[!, col_name] = normalize_column!( df_hourly[!, col_name])
    end
end

df_24h = filter(row -> start_time <= row[:local_15min] < end_time, df_hourly)



cum24hourseachbuspv = []
for i in 1:33
    eachbuspv = df_24h.solar.* dfpv[!, :S][i].*2# Here we multiply each curve (24 hours) by a single value
    push!(cum24hourseachbuspv,eachbuspv)
end

cumbuseachhourpv = []
for i in 1:24
    eachhourpv = df_24h.solar[i].* dfpv[!, :S].*2 # Here we scale the system capacity for all 33 buses at one specific hour by multiplying the system capacity for all 33 buses by the amount of generated pv (curve) at one specific time. then we repeat it 24 hours for all hours. so we have 24 33-dimensional vectors.
    push!(cumbuseachhourpv,eachhourpv)
end    
cumbuseachhourpv[10][:]

cumbuseachhourpv = cumbuseachhourpv.* 1e3 ./ Sbase


#Function to adjust and format numbers, keeping them as numbers
function adjust_and_format_number(num)
    if abs(num) < 1e-4
        return 0.0
    else
        return round(num, digits=4)
    end
end
cumbuseachhourpv = map(sublist -> map(adjust_and_format_number, sublist), cumbuseachhourpv)
#Apply the same process for other vectors
cumbuseachhour = map(sublist -> map(adjust_and_format_number, sublist), cumbuseachhour)
cumbuseachhourreactiveload = map(sublist -> map(adjust_and_format_number, sublist), cumbuseachhourreactiveload)

In [ ]:
htime= 19
#Convert impedances from Ohms to p.u. - loads from kW to pu - battery data from kw to pu - pv data from kw to pu
dfb_pu = DataFrame()
for row in eachrow(dfb)
    newdfb = (index=row.index, Node = row.Node, Pd=row.Pd .* 1e3.*0.5 ./ Sbase, Qd=row.Qd.*0.5 .* 1e3 ./ Sbase, Vmax=1.1, Vmin=0.9, r=row.r ./(Vbase^2 / Sbase), x=row.x ./(Vbase^2 / Sbase), b= row.b)
    push!(dfb_pu, newdfb)
end

dfl_pu = DataFrame()
for row in eachrow(dfl)
    newdfl = (index=row.index, to = row.to , from=row.from , r=row.r ./  (Vbase^2 / Sbase), x=row.x ./ (Vbase^2 / Sbase), b=row.b, s=row.s, g = row.g)
    push!(dfl_pu, newdfl)
end

dfbat_pu = DataFrame()
for row in eachrow(dfbat)
    newdfbat = (index=row.index, Node = row.Node , bmax=row.bmax .* 1e3 ./ Sbase , pmax=row.pmax .* 1e3 ./ Sbase )
    push!(dfbat_pu, newdfbat)
end

dfpv_pu = DataFrame()
for row in eachrow(dfpv)
    newdfpv = (index=row.index, Node = row.Node , S = row.S .* 1e3.*2 ./ Sbase )
    push!(dfpv_pu, newdfpv)
end


#Bring the data in a more useable form
println(">>> preparing data")
buses = []
for row in eachrow(dfb_pu)
    newbus = (index=row.index, ancestor=[], children=[], pd=row.Pd, qd=row.Qd, Vmax=row.Vmax, Vmin = row.Vmin)
    push!(buses, newbus)
end

lines = []
for row in eachrow(dfl_pu)
    newline = (index=row.index, r=row.r, x=row.x, from=row.to, to=row.from)
    push!(buses[newline.to].ancestor, newline.from)
    push!(buses[newline.from].children, newline.to)
    push!(lines, newline)
end

PVs = []
for row in eachrow(dfpv_pu)
    newPV = (index=row.index, S=row.S)
    push!(PVs, newPV)
end

energy_storage_systems = []
for row in eachrow(dfbat_pu)
    newenergy_storage_system = (index=row.index, Pbmax=row.pmax,Bmax = row.bmax )
    push!(energy_storage_systems, newenergy_storage_system)
end

nbuses = length(buses)
nlines = length(lines)
nPVs = length(PVs)
nenergy_storage_systems = length(energy_storage_systems)

# define A matrix (with substation bus!)
A = zeros(length(lines), length(buses))
for bus in buses
    a = bus.index
    while a != 1
        A[a-1, bus.index] = 1
        a = buses[a].ancestor[1]
    end
end

#Define power generation, load active and reactive power vectors
pl= cumbuseachhour[htime]
ql = cumbuseachhourreactiveload[htime]
pav_pu = cumbuseachhourpv[htime]


#Set substation voltage
u_0 = 1

#Define remaining matrices
diag_r = Diagonal([line.r for line in lines])
diag_x = Diagonal([line.x for line in lines])
R = 2*A'*diag_r*A
B = 2*A'*diag_x*A
a = ones(nbuses) .* u_0 


#Define parameters related to the price
c = fill(10, nbuses) #The price associated with the power consumed by the customers
d= fill(3, nbuses) # The feed-in tariff cost to the utility
e= fill(30, nbuses) # The cost of reactive power injection/absorption from the inverters
f= fill(6,nbuses) # The cost of active power curtailment

In [ ]:
forecast_values = hcat(pav_pu,pl,ql)
clusters= [[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,16,17,18],[19,20,21,22],[23,24,25],[26,27,28,29,30,31,32,33]]
n_buses = num_buses
n_clusters = 5
F=pl./ql
F[1]=0

In [ ]:
Random.seed!(123)
#Function to create samples for vector deltas
function create_delta_samples(clusters, forecast_values, n_samples)
    deltas_samples = []
    #For each cluster, create samples for the delta vector
    for cluster in clusters
        cluster_size = length(cluster)
        delta_size = 3 * cluster_size
        samples = zeros(delta_size, n_samples)

        #Create samples
        for i in 1:n_samples
            for j in 1:cluster_size
                #p_av follows a normal distribution with mean 0 and std deviation of 20% of the forecast
                p_av_forecast = forecast_values[cluster[j], 1]
                p_av_std = 0.2 * p_av_forecast
                p_av_dist = Normal(0, p_av_std)
                p_av_sample = rand(p_av_dist)
                p_av_sample = min.(p_av_sample, dfpv_pu[cluster[j], :S] - pav_pu[cluster[j]])
                p_av_sample = max.(p_av_sample, - pav_pu[cluster[j]])

                #p_l follows a normal distribution with mean 0 and std deviation of 20% of the forecast
                p_l_forecast = forecast_values[cluster[j], 2]
                p_l_std = 0.2 * p_l_forecast
                p_l_dist = Normal(0, p_l_std)
                p_l_sample = rand(p_l_dist)
                p_l_sample = min.(p_l_sample, 1.2 * p_l_forecast - p_l_forecast)
                p_l_sample = max.(p_l_sample, 0.5 * p_l_forecast- p_l_forecast)
                
                if F[cluster[j]] == 0
                    q_l_sample= 0
                else    
                    q_l_sample = (p_l_sample + p_l_forecast) ./ F[cluster[j]]
                end
                samples[(j-1)*3+1, i] = p_av_sample + p_av_forecast
                samples[(j-1)*3+2, i] = p_l_sample + p_l_forecast
                samples[(j-1)*3+3, i] = q_l_sample
            end
        end
        push!(deltas_samples, samples)
    end
    return deltas_samples
end


#Create the samples for vector deltas
n_samples = 25 # Number of samples for each vector delta
deltas_samples = create_delta_samples(clusters, forecast_values, n_samples)

In [ ]:
function create_delta_max(clusters, dfpv_pu,pl,ql)
    deltas_max = []
    for cluster in clusters
        cluster_size = length(cluster)
        delta_size = 3 * cluster_size
        max = zeros(delta_size, 1)
        for i in 1:1
            for j in 1:cluster_size
                p_av_max =  dfpv_pu[cluster[j], :S]
                p_l_max=  1.2*pl[cluster[j]]
                if F[cluster[j]]==0
                    q_l_max = 0
                else    
                    q_l_max = 1.2*pl[cluster[j]]/F[cluster[j]]
                end    
                max[(j-1)*3+1, i] = p_av_max
                max[(j-1)*3+2, i] = p_l_max
                max[(j-1)*3+3, i] = q_l_max
            end
        end
        push!(deltas_max, max)
    end
    return deltas_max
end
deltas_max = create_delta_max(clusters, dfpv_pu,pl,ql)

In [ ]:
function create_delta_min(clusters, dfpv_pu,pl,ql)
    deltas_min = []
    for cluster in clusters
        cluster_size = length(cluster)
        delta_size = 3 * cluster_size
        min = zeros(delta_size, 1)
        for i in 1:1
            for j in 1:cluster_size
                p_av_min = 0
                p_l_min=  0.5*pl[cluster[j]]
                if F[cluster[j]]==0
                    q_l_min = 0
                else    
                    q_l_min = 0.5*pl[cluster[j]]/F[cluster[j]]
                end 
                min[(j-1)*3+1, i] = p_av_min
                min[(j-1)*3+2, i] = p_l_min
                min[(j-1)*3+3, i] = q_l_min
            end
        end
        push!(deltas_min, min)
    end
    return deltas_min
end
deltas_min = create_delta_min(clusters, dfpv_pu,pl,ql)
deltas_hat = deltas_samples

In [ ]:
generalobj1 = []
    # For each cluster, create samples for the delta vector
    for cluster in clusters
        cluster_size = length(cluster)
        delta_size = 3 * cluster_size
        general = zeros(1,delta_size)
        # Create samples
        for i in 1:1
            for j in 1:cluster_size
                p_av_min = 0
                p_l_min=  0
                q_l_min = 0

                # Add samples to the corresponding positions in the delta vector
                general[1,(j-1)*3+1] = p_av_min
                general[1,(j-1)*3+2] = p_l_min
                general[1,(j-1)*3+3] = q_l_min
            end
        end
        push!(generalobj1, general)
    end

<b>Run experiment</b> 

In [ ]:
#PV location
pvuncer_loc= [3,5,6,8,11,12,14,16, 17,18,19,21,22,23,25,27,29,31,33] 
#Build optimization model
    function run_optimization(epsilon)
    n_clusters = 5
    ϵ = zeros(Float64, n_clusters)
    for i in 1:n_clusters
        ϵ[i] = epsilon
    end 
    model = Model(SCS.Optimizer)
#Constraint violation probabilities
    eta1 =0.05
    eta=0.05
#Parameters
    n_clusters = 5
    N =nbuses
    D = n_clusters
    o= n_clusters
    Nprime = n_samples
    Vmax=1.1
    Vmin=0.9
    er=[]
    for j in 1:n_clusters
        er1= length(clusters[j])
        push!(er,er1)
    end    
    maximum(er)# the number of buses in the cluster which has the maximum buses
    
#Variables
    @variable(model,0 <= alpha[n=1:nbuses] <= 1) #variable for fraction of active power curtailed by RES
    @variable(model, qc[n=1:nPVs] ) #variable for reactive power provided by RES
    @variable(model, 0 <= pb[n=1: nenergy_storage_systems]) #variable for rate of (dis)charge for energy storage 
    @variable(model, y1[n=1:nPVs] >=0)
    @variable(model, y2[n=1:nPVs] >=0)
    #Auxillary variables for wc exp. cost reformulation
    @variable(model, λ_co[f=1:D] >=0)
    @variable(model, s_co1[n=1:N, i = 1:Nprime] >=0)
    @variable(model, s_co2[n=1:N, i = 1:Nprime] >=0)
    #Auxillary variables for CVaR
    @variable(model, varphi <=0)
    @variable(model, gamma[n=1:N] <=0)
    @variable(model, varpi) 
    @variable(model, psi[n=1:N]) 
    #Auxillary variables
    @variable(model, λ_cc[f=1:D] >=0) #related to constraint v[t] >= buses[t].Vmin and  v[t] =< buses[t].Vmax
    @variable(model, λ_third_cc[n=1:N ] >=0) # related to constraint ((1-alpha[PV.index])*pavs[PV.index])^2 + (qc[PV.index])^2 <= (PV.S)^2
    @variable(model, P_cc[i=1:Nprime]>=0 )
    @variable(model, P[k=1:2*nbuses+1,f=1:D, i=1:Nprime] >=0)
    @variable(model, z[ k=1:2*nbuses+1,f=1:D, i=1:Nprime])
    @variable(model, u[ k=1:2*nbuses+1, f=1:D, m=1:3*length(clusters[f]), i=1:Nprime] >=0)
    @variable(model, l[ k=1:2*nbuses+1, f=1:D, m=1:3*length(clusters[f]), i=1:Nprime] >=0)
    @variable(model, Z_cc[n=1:N, i=1:Nprime]>=0)
    
#Define matrices
        A = []
        for cluster in clusters
            cluster_size = length(cluster)
            delta_size = 3 * cluster_size
            aa = Array{AffExpr}(undef, nbuses, delta_size)
            for n in 1:nbuses
                for j in 1:cluster_size 
                    first =  R[n,cluster[j]]*(1-alpha[cluster[j]])
                    second=  -R[n,cluster[j]]
                    third = -B[n,cluster[j]]
                    # Add elements to the corresponding positions in the matrix A
                    aa[n,(j-1)*3+1] = first
                    aa[n,(j-1)*3+2] = second
                    aa[n,(j-1)*3+3] = third
                end
            end
            push!(A, aa)
        end

        AJ=vcat(A,-A,generalobj1)
        a_prime = Array{AffExpr}(undef, 2*nbuses+1, n_clusters, 3*maximum(er))
        for i in 1:(2*nbuses+1)
            for j in 1:n_clusters
                for k in 1:3*maximum(er)
                    a_prime[i, j, k] = @expression(model, 0)
                end
            end
        end
        for b in 1:length(AJ)
            for c in 1:n_buses
                if b < n_clusters+1
                    if 3*length(clusters[b]) == 3*maximum(er)
                        for d in 1:3*maximum(er)
                            a_prime[c,b,d] = AJ[b][c,:][d]
                        end   
                    else
                        for d in 1:3*maximum(er)
                            if d < 3*length(clusters[b])+1
                                a_prime[c,b,d] = AJ[b][c,:][d]
                            else
                                a_prime[c,b,d]=0
                            end    
                        end    
                    end
                elseif b > n_clusters && b < 2*n_clusters+1 
                    if 3*length(clusters[b - n_clusters]) == 3*maximum(er)
                         for d in 1:3*maximum(er)
                             a_prime[c + nbuses,b - n_clusters,d] = AJ[b][c,:][d]
                         end   
                    else
                        for d in 1:3*maximum(er)
                            if d < 3*length(clusters[b - n_clusters])+1
                                a_prime[c + nbuses,b - n_clusters,d] = AJ[b][c,:][d]
                            else
                                 a_prime[c + nbuses,b - n_clusters,d]=0
                            end    
                         end  
                    end    
                else
                    if c == 1
                        if 3*length(clusters[b - 2*n_clusters]) == 3*maximum(er)
                            for d in 1:3*maximum(er) 
                                a_prime[c + 2*nbuses,b - 2*n_clusters,d] = AJ[b][c,:][d]
                            end    
                        else
                             for d in 1:3*maximum(er)
                                if d < 3*length(clusters[b - 2*n_clusters])+1
                                     a_prime[c + 2*nbuses,b - 2*n_clusters,d] =  AJ[b - 2*n_clusters][c,:][d]
                                else
                                    a_prime[c + 2*nbuses,b - 2*n_clusters,d]=0
                                end    
                             end           
                         end
                    end
                 end
             end
        end    


#Expressions
    @expression(model, b_first[t=1:nbuses], -R[t,:]'*pb +B[t,:]'*qc +a[t] - Vmax -varphi)
    @expression(model, b_second[t=1:nbuses], R[t,:]'*pb - B[t,:]'*qc -a[t] + Vmin -varphi)
    @expression(model, b_third_1[t=1:nbuses], qc[t]^2 - dfpv_pu[t, :S]^2 - gamma[t])
    b_third_0= zeros(Float64, nbuses)
    @expression(model, voltage[n=1:nbuses], sum(dot(A[f][n,:], deltas_samples[f][:,1]) for f in 1:D) -R[n,:]'*pb +B[n,:]'*qc +a[n])# the definition of voltage in our model
    @expression(model, c_prime[i=1:2*nbuses+1], 
    if i <= nbuses
        b_first[i]  # The first 33 elements from b_first
    elseif i <= 2 * nbuses
        b_second[i - nbuses]  # The next 33 elements from b_second
    else
        0  # The 67th element is zero
    end
)
    
#Constraints    
    @constraint(model,absolute[n=1:N] , qc[n] == y1[n] - y2[n]) # Resolving the absolute value of qc[n] in the objective function
    #Deterministic constraints
    for energy_storage_system in energy_storage_systems
    @constraint(model, pb[energy_storage_system.index] <= energy_storage_system.Pbmax) #Upper limit constraint for storage device
    @constraint(model, pb[energy_storage_system.index] >= -energy_storage_system.Pbmax)#Lower limit constraint for storage device
    end
    #CVaR reformulation of chance constraints (voltage constraints)
    @constraint(model, 0 >=varpi + varphi)
    @constraint(model,cvar_third[n=1:N],0 >= psi[n] + gamma[n])
    constraint_ref = @constraint(model, eta1*varpi >= sum(λ_cc[f] * ϵ[f] for f in 1:D) + (1/Nprime) * sum(P_cc[i] for i in 1:Nprime))
    @constraint(model, DD1[k=1:2*nbuses+1, i=1:Nprime], P_cc[i] >= c_prime[k] + sum(P[k,f,i] for f in 1:D))
    @constraint(model, rhof1_up[k=1:2*nbuses+1, f=1:D, i=1:Nprime], P[k,f,i] >=  sum( z[k,f,i]*deltas_hat[f][m,:][i] + (u[k,f,m,i] * deltas_max[f][m] -l[k,f,m,i]* deltas_min[f][m]) for m in 1:3*length(clusters[f])))  
    @constraint(model, rhof1_lo[k=1:2*nbuses+1, f=1:D, m=1:3*length(clusters[f]), i=1:Nprime], a_prime[k,f,m]-z[k,f,i] == u[k,f,m,i]- l[k,f,m,i])
    @constraint(model, rhof1_av[k=1:2*nbuses+1, f=1:D, i=1:Nprime],    z[k,f,i] >= -λ_cc[f])
    @constraint(model, rhof1_av2[k=1:2*nbuses+1, f=1:D, i=1:Nprime],  z[k,f,i] <= λ_cc[f])

#Inverter constraint reformulation 
    for n in 1:nbuses
        if dfpv_pu[n, :S] == 0.0
            @constraint(model, alpha[n]==0)
        end
    end
    constraint_refs = []
    for n in 1:nbuses
        if dfpv_pu[n, :S] != 0.0
            for cluster in clusters
                s_r= zeros(1,3*length(cluster))
                for j in 1:length(cluster)
                    if cluster[j] == n
                        indxofcluster = findfirst(x -> x == cluster, clusters)
                        s_r[1,(j-1)*3+1]=1
                        main_constraint_ref = @constraint(model,  eta * psi[n] >= λ_third_cc[n] * ϵ[indxofcluster]  + (1/Nprime) * sum(Z_cc[n,i] for i in 1:Nprime))
                        for i in 1:Nprime
                        @constraint(model, Z_cc[n,i] >= b_third_1[n] + ((1-alpha[n])*dot(s_r,deltas_max[indxofcluster]))^2 - λ_third_cc[n]*(dot(s_r,deltas_max[indxofcluster])-(s_r*deltas_hat[indxofcluster])[i]))
                        @constraint(model, Z_cc[n,i] >= b_third_1[n] + ((1-alpha[n])*(s_r*deltas_hat[indxofcluster])[i])^2)
                        @constraint(model, Z_cc[n,i] >= 0)
                        end
                        push!(constraint_refs, main_constraint_ref)
                    end
                 end
             end 
        end   
     end

#Objective reformulation
    obj_refs = []
    obj_refs2=[]
    for n in 1:nbuses
            totaltotal = 0 
            for cluster in clusters
                for j in 1:length(cluster)
                    if cluster[j] == n
                        r_n= zeros(1,3*length(cluster))
                        m_n= zeros(1,3*length(cluster))
                        indxofcluster = findfirst(x -> x == cluster, clusters)
                        m_n[1,(j-1)*3+1]=1
                        r_n[1,(j-1)*3+2]=1
                        total1 = 0
                        total2=0
                        for i in 1:Nprime
                        @constraint(model, s_co1[n,i] >= c[n]*(dot(r_n,deltas_max[indxofcluster]) - (1-alpha[n])* (dot(m_n,deltas_min[indxofcluster])) + pb[n]) - λ_co[indxofcluster]*((dot(r_n,deltas_max[indxofcluster]) - (r_n*deltas_hat[indxofcluster])[i]) - (dot(m_n,deltas_min[indxofcluster]) - (m_n*deltas_hat[indxofcluster])[i])) )
                        @constraint(model, s_co1[n,i] >= d[n]*(-dot(r_n,deltas_min[indxofcluster]) + (1-alpha[n])* (dot(m_n,deltas_max[indxofcluster])) - pb[n]) - λ_co[indxofcluster]*((- dot(r_n,deltas_min[indxofcluster]) + (r_n*deltas_hat[indxofcluster])[i]) + (dot(m_n,deltas_max[indxofcluster]) - (m_n*deltas_hat[indxofcluster])[i])))
                        @constraint(model, s_co1[n,i] >= c[n]* ((r_n*deltas_hat[indxofcluster])[i]- (1-alpha[n])*(m_n*deltas_hat[indxofcluster])[i] + pb[n]))
                        @constraint(model, s_co1[n,i] >= d[n]* (-(r_n*deltas_hat[indxofcluster])[i]+ (1-alpha[n])*(m_n*deltas_hat[indxofcluster])[i] - pb[n]))
                        @constraint(model, s_co1[n,i] >= 0)
                        @constraint(model, s_co2[n,i] >= f[n]*alpha[n]*(dot(m_n,deltas_max[indxofcluster])) - λ_co[indxofcluster]*(dot(m_n,deltas_max[indxofcluster])- (m_n*deltas_hat[indxofcluster])[i] ))
                        @constraint(model, s_co2[n,i] >= f[n]*alpha[n]*(dot(m_n,deltas_min[indxofcluster])) + λ_co[indxofcluster]*(dot(m_n,deltas_min[indxofcluster])- (m_n*deltas_hat[indxofcluster])[i] ))
                        @constraint(model, s_co2[n,i] >= f[n]*alpha[n]* ((m_n*deltas_hat[indxofcluster])[i]))
                        @constraint(model, s_co2[n,i] >= 0)
                        total1 += s_co1[n,i]
                        total2 += s_co2[n,i]
                        end
                        push!(obj_refs, (1/Nprime) * total1)
                        push!(obj_refs2, (1/Nprime) * total2)
                    end
                 end
             end  
     end
     
#Objective function
    expcost = sum(λ_co[f]*ϵ[f] for f in 1:D)  + sum( obj_refs[n] + obj_refs2[n]  for n in 1:N) + sum(( e[n]* (y1[n]+y2[n])) for n in 1:N )
    @objective(model, Min,expcost)
    
#Solve optimization model
    optimize!(model)
    obj_value = JuMP.objective_value(model)
    v_opt = [round(x < 1e-4 ? 0.0 : x, digits=4) for x in value.(model[:voltage])]
    qc_opt = [round(x, digits=4) for x in value.(model[:qc])]
    alpha_opt = [round(x < 1e-4 ? 0.0 : x, digits=4) for x in value.(model[:alpha])]
    pb_opt = [round(x < 1e-4 ? 0.0 : x, digits=4) for x in value.(model[:pb])]
    λ_cc_opt = [round(x < 1e-4 ? 0.0 : x, digits=4) for x in value.(model[:λ_cc])]
    λ_third_cc_opt = [round(x < 1e-4 ? 0.0 : x, digits=4) for x in value.(model[:λ_third_cc])[pvuncer_loc]]
    λ_cost_opt = [round(x < 1e-4 ? 0.0 : x, digits=4) for x in value.(model[:λ_co])]
    obj_value_opt = round(value.(obj_value), digits=4) 
    dual_value_cc = dual(constraint_ref)
    dual_value_inv=[]
    #Define dual variables
    for ref in constraint_refs
        dual_va=zeros(length(pvuncer_loc),1)
        dual_va = dual(ref)
        push!(dual_value_inv,dual_va)
    end
    return v_opt,qc_opt, alpha_opt, pb_opt, λ_cc_opt, λ_third_cc_opt, λ_cost_opt, obj_value_opt,dual_value_cc,dual_value_inv
end

<b>Result</b> 

In [ ]:
#Save results
epsilon_values=[1,0.1,0.01,0.005,0.001,0.0001]
for epsilon in epsilon_values
    result=run_optimization(epsilon)
    df_variables = DataFrame(  voltage= result[1], qc=  result[2], alpha=  result[3], pb= result[4])
    df_λ_cc= DataFrame(λ_cc= result[5])
    df_λ_third_cc= DataFrame(λ_third_cc =  result[6]) 
    df_λ_cost= DataFrame(λ_cost= result[7])
    df_objective= DataFrame(objective= result[8])
    df_dualcc= DataFrame(dualcc= result[9])
    df_dualinv= DataFrame(dualinv= result[10])
end